### attribute_00.csv 안의 내용이랑 이름이 매칭이 안되고 있음

- 직접 추출하든 load_dataset을 바꾸든 해야함.

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

def scan_and_collect_files(datasets_dir):
    """
    주어진 데이터셋 디렉토리를 스캔하여 각 클래스의 train 및 test 디렉토리 내의 모든 WAV 파일을 수집합니다.

    Args:
        datasets_dir (str): 데이터셋의 루트 디렉토리 경로 (예: '../../datasets/dev').

    Returns:
        dict: 클래스 이름을 키로 하고, 해당 클래스의 WAV 파일 경로 리스트를 값으로 하는 딕셔너리.
    """
    data = {}

    # 데이터셋 디렉토리 내의 모든 클래스 디렉토리를 순회
    for class_name in os.listdir(datasets_dir):
        class_dir = os.path.join(datasets_dir, class_name)
        if not os.path.isdir(class_dir):
            continue  # 디렉토리가 아닌 경우 건너뜀

        data[class_name] = []

        # 'train'과 'test' 디렉토리 경로 설정
        train_dir = os.path.join(class_dir, 'train')
        test_dir = os.path.join(class_dir, 'test')

        # 'train' 디렉토리 내의 WAV 파일 수집
        if os.path.isdir(train_dir):
            for file in os.listdir(train_dir):
                if file.lower().endswith('.wav'):
                    # 파일 경로를 'datasets/dev' 기준으로 상대 경로로 생성
                    relative_path = os.path.normpath(os.path.join(class_name, 'train', file))
                    data[class_name].append(relative_path)
        else:
            print(f"Warning: 'train' 디렉토리가 '{class_name}' 클래스에 존재하지 않습니다.")

        # 'test' 디렉토리 내의 WAV 파일 수집
        if os.path.isdir(test_dir):
            for file in os.listdir(test_dir):
                if file.lower().endswith('.wav'):
                    # 파일 경로를 'datasets/dev' 기준으로 상대 경로로 생성
                    relative_path = os.path.normpath(os.path.join(class_name, 'test', file))
                    data[class_name].append(relative_path)
        else:
            print(f"Warning: 'test' 디렉토리가 '{class_name}' 클래스에 존재하지 않습니다.")

    return data

def backup_existing_csv(csv_path):
    """
    기존의 CSV 파일을 백업합니다.

    Args:
        csv_path (str): 백업할 CSV 파일의 경로.
    """
    if os.path.exists(csv_path):
        backup_path = csv_path.replace('.csv', '_backup_before_update.csv')
        os.rename(csv_path, backup_path)
        print(f"기존 CSV 파일을 백업했습니다: {backup_path}")
    else:
        print(f"백업할 기존 CSV 파일이 존재하지 않습니다: {csv_path}")

def update_attributes_csv(datasets_dir, data):
    """
    각 클래스 디렉토리에 있는 'attributes_00.csv' 파일을 덮어씌웁니다.
    파일 이름 순으로 정렬하여 저장합니다.

    Args:
        datasets_dir (str): 데이터셋의 루트 디렉토리 경로.
        data (dict): 클래스 이름을 키로 하고, 해당 클래스의 WAV 파일 경로 리스트를 값으로 하는 딕셔너리.
    """
    for class_name, file_list in data.items():
        class_dir = os.path.join(datasets_dir, class_name)
        csv_path = os.path.join(class_dir, 'attributes_00.csv')

        # 기존 CSV 백업 (옵션)
        backup_existing_csv(csv_path)

        # 파일 이름 순으로 정렬
        sorted_file_list = sorted(file_list)

        # 데이터프레임 생성
        df = pd.DataFrame({'file_name': sorted_file_list})

        # CSV 파일로 저장
        df.to_csv(csv_path, index=False)
        print(f"'{csv_path}' 파일을 성공적으로 업데이트했습니다. 총 {len(sorted_file_list)}개의 파일이 기록되었습니다.")

def main():
    # 데이터셋 디렉토리 경로 설정
    datasets_dir = "../../datasets/dev"

    # 데이터셋 스캔 및 파일 수집
    print("데이터셋 디렉토리를 스캔하고 WAV 파일을 수집하는 중...")
    data = scan_and_collect_files(datasets_dir)

    # 파일 존재 여부 확인 및 필터링 (선택 사항)
    print("\n수집된 파일의 존재 여부를 확인하는 중...")
    for class_name, file_list in data.items():
        existing_files = []
        for file_path in tqdm(file_list, desc=f"Checking files for class '{class_name}'"):
            full_path = os.path.join(datasets_dir, file_path)
            if os.path.exists(full_path):
                existing_files.append(file_path)
            else:
                print(f"Warning: 파일이 존재하지 않습니다: {full_path}")
        data[class_name] = existing_files

    # CSV 파일 업데이트
    print("\n'attributes_00.csv' 파일을 업데이트하는 중...")
    update_attributes_csv(datasets_dir, data)

    print("\n모든 클래스의 'attributes_00.csv' 파일이 성공적으로 업데이트되었습니다.")

if __name__ == "__main__":
    main()
